In [2]:
import numpy as np
import pandas as pd
import re
from datetime import datetime as dt

In [3]:
url = 'https://raw.githubusercontent.com/hkfuoc98/Imaginary-Data-Projects/master/Case%20Ecommerce%20Logistics.csv'
df = pd.read_csv(url)
df['time'] = pd.to_datetime(df['time'],dayfirst = True,utc=True)
df.head()

,order number,time,event
0,#1,2022-07-14 14:41:00+00:00,Successful delivery. Receiver: XXX
1,#1,2022-07-14 14:35:00+00:00,Being delivered
2,#1,2022-07-14 12:39:00+00:00,"Unsuccessful delivery, receiver suggests anoth..."
3,#1,2022-07-14 08:57:00+00:00,Being delivered
4,#1,2022-07-14 05:23:00+00:00,Has been delivered to warehouse 50-HCM Quan 10...


In [5]:
def return_mean_med_1st_delivery(df,startstring,deliverstring1,deliverstring2):
    start = df[df['event']==startstring].loc[:,['order number','time']].set_index('order number')
    
    first_delivery = df[df['event'].str.startswith(deliverstring1) | df['event'].str.startswith(deliverstring2)].loc[:,['order number','time']]
    first_delivery.rename(columns = {'order number':'order number','time':'first delivery'},inplace = True)
    first_delivery = first_delivery.groupby(by=['order number'])['first delivery'].min()
    
    to_first_deli = start.join(first_delivery)
    to_first_deli['hrs taken']=to_first_deli['first delivery']-to_first_deli['time']
    mean = to_first_deli['hrs taken'].mean()
    med = to_first_deli['hrs taken'].median()
    return 'mean of 1st time: '+ str(mean) + ' and median of 1st time: '+ str(med)

return_mean_med_1st_delivery(df,'Has been delivered to warehouse Cu Chi SOC','Unsuccessful delivery','Successful delivery')

'mean of 1st time: 0 days 20:35:00 and median of 1st time: 0 days 17:46:00'

'mean of 1st time: 0 days 20:35:00 and median of 1st time: 0 days 17:46:00'

In [7]:
def return_order_not_effective(df,wantedhub):
    # Lọc các đơn không thông qua hub: 50-HCM Quan 10/Phuong 15 LM Hub
    effective = df[df['event'].str.contains(wantedhub)].set_index('order number')
    noteffective = df[~df['order number'].isin(effective.index)]
    return pd.unique(noteffective['order number'])
return_order_not_effective(df,'50-HCM Quan 10/Phuong 15 LM Hub')

array(['#3', '#4'], dtype=object)